## Connect database

In [16]:
import psycopg2

user = input('user: ')

password = input('password: ')

user = user
password = password
host = 'capably-replete-rudd.data-1.use1.tembo.io'
port = '5432'
database = 'barber_shop'
ssl_cert= '../configs/ca.crt'

In [17]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}?sslmode=verify-full&sslrootcert={ssl_cert}')

### Extract ddl models

In [10]:
import pandas as pd

In [18]:
base_content_query = '''
SELECT 
    table_name,
    column_name,
    data_type
FROM 
    information_schema.columns
WHERE 
    table_schema = 'dw'
ORDER BY 
    table_name, column_name;
'''

base_content = pd.read_sql(base_content_query, engine)

base_content

,table_name,column_name,data_type
0,dim_cliente,id_origem_cliente,integer
1,dim_cliente,nm_cliente,character varying
2,dim_cliente,sk_cliente,integer
3,dim_funcionario,id_origem_funcionario,integer
4,dim_funcionario,nm_funcionario,character varying
5,dim_funcionario,sk_funcionario,integer
6,dim_pagamento,sk_pagamento,integer
7,dim_pagamento,tp_pagamento,character varying
8,dim_servico,sk_servico,integer
9,dim_servico,tp_servico,character varying


In [20]:
ddl_schemas = []

for table_name in base_content['table_name'].unique():
    table_columns = base_content[base_content['table_name'] == table_name]
    
    ddl = f"CREATE TABLE dw.{table_name} (\n"
    columns = []
    
    for _, row in table_columns.iterrows():
        column = f"    {row['column_name']} {row['data_type']}"
        columns.append(column)
    
    ddl += ",\n".join(columns)
    ddl += "\n);"
    
    ddl_schemas.append(ddl)

ddl_text = "\n\n".join(ddl_schemas)

print(ddl_text)

CREATE TABLE dw.dim_cliente (
    id_origem_cliente integer,
    nm_cliente character varying,
    sk_cliente integer
);

CREATE TABLE dw.dim_funcionario (
    id_origem_funcionario integer,
    nm_funcionario character varying,
    sk_funcionario integer
);

CREATE TABLE dw.dim_pagamento (
    sk_pagamento integer,
    tp_pagamento character varying
);

CREATE TABLE dw.dim_servico (
    sk_servico integer,
    tp_servico character varying
);

CREATE TABLE dw.fact_servicos_barbearia (
    av_comentario character varying,
    av_servico integer,
    dat_comp date,
    dat_servico timestamp without time zone,
    fact_servicos_barbearia_id integer,
    sk_cliente integer,
    sk_funcionario integer,
    sk_pagamento integer,
    sk_servico integer,
    vl_servico numeric
);


In [31]:
import google.generativeai as genai

gemini_api = input('api key: ')

gen_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain"
}

prompt = f"""
Você é um assistente de análise de dados, com o objetivo de extrair dados de uma base postgres SQL sob a demanda.
A base com a qual está trabalhando no momento possui o seguinte esquema:

<esquema>
{ddl_text}
</esquema>

<descricao_tabelas>
A tabela dim_cliente contém dados que identificam os clientes
A tabela dim_funcionario contém dados que identificam os funcionários
A tabela dim_pagamento contém dados que identificam as formas de pagamento disponíveis
A tabela dim_servico contém dados que identificam os serviços disponíveis
A tabela fact_servicos_barbearia contém dados de cada atendimento prestado
</descricao_tabelas>

<instruções>
1 - Antes de gerar a query, faça:
    - Consulte <descricao_tabelas></descricao_tabelas> para entender do que se trata cada tabela
2 - Com base unicamente no <esquema></esquema> fornecido, gere uma query sql que responda a pergunta do usuário;
3 - Otimize a query gerada para a melhor performance;
4 - Mantenha boas práticas em SQL;
5 - Nunca traga resultados com IDs;
5 - Retorne apenas a query gerada.
</instruções>
"""

genai.configure(api_key = gemini_api)

model = genai.GenerativeModel(
    model_name= 'gemini-1.5-pro',
    generation_config= gen_config,
    system_instruction= prompt)

In [32]:
response = model.generate_content('Quais são os 5 clientes mais insatisfeitos?')

print(response.text)

```sql
SELECT nm_cliente
FROM dw.fact_servicos_barbearia AS f
JOIN dw.dim_cliente AS c ON f.sk_cliente = c.sk_cliente
WHERE av_servico IS NOT NULL -- Considerando que a avaliação de serviço NULL significa que o cliente não avaliou
ORDER BY av_servico ASC
LIMIT 5;
```

